In [2]:
import numpy as np
import cupy as cp

In [3]:
position_grid = np.linspace(-2, 100, 6001)

In [16]:
from emerald.quantum.msc_unperturbed import MsC_hamiltonian, MsC_eigstates
H_cpu, position_grid = MsC_hamiltonian(0.5, position_grid)

In [17]:
energies_cpu, vectors_cpu = MsC_eigstates(H_cpu, position_grid, 200)

In [10]:
from emerald.quantum.utils import interaction_matrix, wavefunction_stationary_evolution

In [8]:
Xi_vectors, Xi_values, Xi_inv = interaction_matrix(position_grid, vectors_cpu)

In [19]:
print(position_grid.shape)
print(vectors_cpu.shape)
print(Xi_vectors.shape)
print(energies_cpu.shape)

(6001,)
(6001, 200)
(200, 200)
(200,)


In [11]:
wavefunc_history = wavefunction_stationary_evolution(np.linspace(0, 1, 100), energies_cpu, vectors_cpu[:, 0])

  0%|          | 0/99 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (200,) (6001,) 

In [37]:
import cupyx

In [38]:
import cupyx.scipy.linalg


def MsC_potential_vec_cuda(alpha: float, r: cp.ndarray) -> cp.ndarray:
    """Calculates the 1D Morse-Coulomb potential for an array of positions r."""
    D = 1 / alpha
    beta = 1 / (alpha * cp.sqrt(2))
    
    coulomb = -1 / cp.sqrt(r * r + alpha * alpha)
    morse = D * (cp.exp(-2 * beta * r) - 2 * cp.exp(-beta * r))
    
    return cp.where(r > 0, coulomb, morse)

def MsC_hamiltonian_cuda(alpha, position_grid):
    # Grid setup
    L = position_grid[-1] - position_grid[0]
    delta_r = position_grid[1] - position_grid[0]
    
    N = int(cp.ceil(L / delta_r))
    N += int(N % 2 == 0)  # Ensure N is odd
    
    r_grid = cp.linspace(position_grid[0], position_grid[-1], N)
    n = (N - 1) // 2

    # Compute the kinetic energy first row
    l = cp.arange(n)
    g = 2 * (np.pi * l / L) ** 2  # g[0] = 0 naturally
    theta = 2 * np.pi / (N - 1)
    k = cp.arange(N)
    C = cp.cos(l[:, None] * k * theta)  # Shape: (n, N)
    t = (2 / (N - 1)) * cp.dot(g, C)  # First row of T

    # Construct Hamiltonian
    T = cupyx.scipy.linalg.toeplitz(t)  # Symmetric Toeplitz matrix
    V = cp.diag(MsC_potential_vec_cuda(alpha, r_grid))  # Diagonal potential
    H = T + V

    return H, r_grid

In [41]:
def MsC_eigstates_cuda(H, position_grid):

    delta_r = position_grid[1] - position_grid[0]

    # Eigendecomposition
    eig_energies, eig_states = cp.linalg.eigh(H)
    sort_indexes = cp.argsort(eig_energies)
    eig_energies = eig_energies[sort_indexes]
    eig_states = eig_states[:, sort_indexes] / cp.sqrt(delta_r)

    return eig_energies, eig_states

In [39]:
H_gpu, position_grid_gpu = MsC_hamiltonian_cuda(0.5, position_grid)

In [42]:
energies_gpu, vectors_gpu = MsC_eigstates_cuda(H_gpu, position_grid)

In [44]:
print(energies_cpu)
print(energies_cpu-cp.asnumpy(energies_gpu))

[-8.47826445e-01 -1.77874186e-01 -7.06029435e-02 ...  1.70622495e+04
  1.70640259e+04  1.88557289e+04]
[-3.40072415e-12 -4.68242112e-12 -4.57918425e-12 ...  3.63797881e-12
  0.00000000e+00  0.00000000e+00]


In [40]:
print(np.max( np.abs(H_cpu-cp.asnumpy(H_gpu) )))

5.002220859751105e-12


In [28]:
x_gpu = cp.asarray(x_cpu)

In [29]:
w_gpu, v_gpu = cp.linalg.eigh(x_gpu)

OutOfMemoryError: Out of memory allocating 2,056,392,192 bytes (allocated so far: 1,320,000,000 bytes).

In [45]:
mempool = cp.get_default_memory_pool()
mempool.free_all_blocks()

In [46]:
pinned_mempool = cp.get_default_pinned_memory_pool()
pinned_mempool.free_all_blocks()